# BindCraftRun: Local Binder Design Starter
Use this notebook to configure and launch BindCraft locally. It assumes your input structures live in `../InputTargets/` and that results should be written under `../Results/`.

## Prerequisites
- BindCraft repository cloned and `install_bindcraft.sh` completed.
- Notebook running inside the `BindCraft` conda environment created by the installer.
- AlphaFold2 weights downloaded during installation (default under `params/`).
- CUDA-compatible NVIDIA GPU with drivers matching the selected CUDA runtime.
- Target PDB files available in `../InputTargets/` (adjust the variables below if you store them elsewhere).

In [12]:
from pathlib import Path
import json
import subprocess
import shlex
import os

BINDCRAFT_ROOT = Path(r'/mnt/e/Code/BindCraft').resolve()
BINDCRAFT_SCRIPT = BINDCRAFT_ROOT / 'bindcraft' / 'bindcraft.py'

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

print(f"BindCraft root: {BINDCRAFT_ROOT}")
if 'CONDA_DEFAULT_ENV' in os.environ:
    print(f"Active conda env: {os.environ['CONDA_DEFAULT_ENV']}")
else:
    print('CONDA_DEFAULT_ENV not set; confirm you activated the BindCraft environment.')


BindCraft root: /mnt/e/Code/BindCraft
Active conda env: BindCraft


In [13]:
# Confirm the GPU that BindCraft will see
try:
    subprocess.run(['nvidia-smi'], check=True)
except FileNotFoundError:
    print('nvidia-smi not found. Ensure the NVIDIA drivers are installed and visible in this environment.')
except subprocess.CalledProcessError as exc:
    print('nvidia-smi failed with exit code', exc.returncode)


Thu Oct 16 15:05:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77.01              Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     On  |   00000000:01:00.0  On |                  N/A |
| 72%   50C    P2             81W /  285W |   10861MiB /  12282MiB |     39%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Configure target and output paths
Update the variables below to point at your desired settings file, target PDB, and output directory. The defaults use the example `8Z8L.pdb` located in `../InputTargets/`.

In [14]:
# Core project defaults for a quick P51679 run
BINDCRAFT_ROOT = Path(r'/mnt/e/Code/BindCraft').resolve()
BINDCRAFT_SCRIPT = BINDCRAFT_ROOT / 'bindcraft' / 'bindcraft.py'
SETTINGS_DIR = BINDCRAFT_ROOT / 'bindcraft' / 'settings_target'
FILTERS_DIR = BINDCRAFT_ROOT / 'bindcraft' / 'settings_filters'
ADVANCED_DIR = BINDCRAFT_ROOT / 'bindcraft' / 'settings_advanced'

DESIGN_NAME = 'P51679_quick'
SETTINGS_TEMPLATE = SETTINGS_DIR / 'P51679_template.json'
FILTERS_JSON = FILTERS_DIR / 'peptide_relaxed_filters.json'
ADVANCED_JSON = ADVANCED_DIR / 'P51679_quick.json'

TARGETS_DIR = (BINDCRAFT_ROOT / 'InputTargets').resolve()
TARGET_PDB_NAME = 'P51679.pdb'
TARGET_CHAIN = 'A'
HOTSPOT_RESIDUES = None
TARGET_PDB = (TARGETS_DIR / TARGET_PDB_NAME).resolve()
if not TARGET_PDB.exists():
    raise FileNotFoundError(f'Target PDB {TARGET_PDB} not found. Update TARGET_PDB_NAME or TARGETS_DIR.')

RESULTS_BASE = (BINDCRAFT_ROOT / 'Results').resolve()
RESULTS_BASE.mkdir(parents=True, exist_ok=True)
DESIGN_OUTPUT = RESULTS_BASE / DESIGN_NAME
DESIGN_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f'Design name       : {DESIGN_NAME}')
print(f'Settings template : {SETTINGS_TEMPLATE}')
print(f'Filters file      : {FILTERS_JSON}')
print(f'Advanced file     : {ADVANCED_JSON}')
print(f'Targets directory : {TARGETS_DIR}')
print(f'Target PDB        : {TARGET_PDB}')
print(f'Target chain      : {TARGET_CHAIN}')
print(f'Hotspot residues  : {HOTSPOT_RESIDUES}')
print(f'Design output dir : {DESIGN_OUTPUT}')


Design name       : P51679_quick
Settings template : /mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_template.json
Filters file      : /mnt/e/Code/BindCraft/bindcraft/settings_filters/peptide_relaxed_filters.json
Advanced file     : /mnt/e/Code/BindCraft/bindcraft/settings_advanced/P51679_quick.json
Targets directory : /mnt/e/Code/BindCraft/InputTargets
Target PDB        : /mnt/e/Code/BindCraft/InputTargets/P51679.pdb
Target chain      : A
Hotspot residues  : None
Design output dir : /mnt/e/Code/BindCraft/Results/P51679_quick


### Create or reuse a dedicated settings file
Set `CREATE_SETTINGS_COPY = True` the first time you run for a new project. The template (`settings.json`) will be copied to `settings_target/<design name>.json`, with `design_path`, `binder_name`, and `starting_pdb` updated to match your local folders. If the file already exists it will be reused unless you enable `OVERWRITE_SETTINGS`.

In [15]:
CREATE_TEMPLATE = True
OVERWRITE_TEMPLATE = False
TEMPLATE_PATH = SETTINGS_DIR / 'P51679_template.json'

if CREATE_TEMPLATE:
    template_cfg = {
        'design_path': str(DESIGN_OUTPUT),
        'binder_name': 'P51679_quick',
        'starting_pdb': str(TARGET_PDB),
        'chains': TARGET_CHAIN,
        'target_hotspot_residues': HOTSPOT_RESIDUES,
        'lengths': [16, 32],
        'number_of_final_designs': 1
    }
    if TEMPLATE_PATH.exists() and not OVERWRITE_TEMPLATE:
        print(f'{TEMPLATE_PATH} already exists. Reusing it; set OVERWRITE_TEMPLATE = True to regenerate.')
    else:
        with TEMPLATE_PATH.open('w') as fh:
            json.dump(template_cfg, fh, indent=2)
        print(f'Wrote {TEMPLATE_PATH}')
else:
    if not TEMPLATE_PATH.exists():
        raise FileNotFoundError(f'Template {TEMPLATE_PATH} missing. Run with CREATE_TEMPLATE = True first.')

SETTINGS_JSON = TEMPLATE_PATH
CREATE_SETTINGS_COPY = True
OVERWRITE_SETTINGS = False
PERSONAL_SETTINGS_PATH = SETTINGS_DIR / f'{DESIGN_NAME}.json'

print(f'Using template file: {SETTINGS_JSON}')

if CREATE_SETTINGS_COPY:
    with SETTINGS_JSON.open() as fh:
        target_cfg = json.load(fh)
    target_cfg['design_path'] = str(DESIGN_OUTPUT)
    target_cfg['binder_name'] = DESIGN_NAME
    target_cfg['starting_pdb'] = str(TARGET_PDB)
    target_cfg['chains'] = TARGET_CHAIN
    target_cfg['target_hotspot_residues'] = HOTSPOT_RESIDUES
    target_cfg['lengths'] = [16, 32]
    target_cfg['number_of_final_designs'] = 1

    if PERSONAL_SETTINGS_PATH.exists() and not OVERWRITE_SETTINGS:
        print(f'{PERSONAL_SETTINGS_PATH} already exists. Reusing it; set OVERWRITE_SETTINGS = True to regenerate.')
    else:
        with PERSONAL_SETTINGS_PATH.open('w') as fh:
            json.dump(target_cfg, fh, indent=2)
        print(f'Wrote {PERSONAL_SETTINGS_PATH}')

    RUN_SETTINGS = PERSONAL_SETTINGS_PATH
else:
    RUN_SETTINGS = SETTINGS_JSON
    print('Using the template directly; ensure paths are valid for your environment.')

with RUN_SETTINGS.open() as fh:
    preview_cfg = json.load(fh)

print(f'Using settings file: {RUN_SETTINGS}')
preview_keys = ['design_path','binder_name','starting_pdb','chains','target_hotspot_residues','lengths','number_of_final_designs']
for key in preview_cfg:
    if key in preview_cfg:
        print(f'{key:>24}: {preview_cfg[key]}')


/mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_template.json already exists. Reusing it; set OVERWRITE_TEMPLATE = True to regenerate.
Using template file: /mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_template.json
/mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_quick.json already exists. Reusing it; set OVERWRITE_SETTINGS = True to regenerate.
Using settings file: /mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_quick.json
             design_path: /mnt/e/Code/BindCraft/Results/P51679_quick
             binder_name: P51679_quick
            starting_pdb: /mnt/e/Code/BindCraft/InputTargets/P51679.pdb
                  chains: A
 target_hotspot_residues: None
                 lengths: [16, 32]
 number_of_final_designs: 1


### Reduce plot and animation outputs
Set `CREATE_ADVANCED_COPY = True` to write a project-specific advanced settings file that disables high-volume plot/animation outputs while keeping archives zipped. This helps keep the design directory smaller.

In [16]:
CREATE_ADVANCED_COPY = False
OVERWRITE_ADVANCED = False
ADVANCED_COPY_PATH = ADVANCED_DIR / 'P51679_quick.json'

if CREATE_ADVANCED_COPY:
    with ADVANCED_JSON.open() as fh:
        advanced_cfg = json.load(fh)
    advanced_cfg.update({})
    if ADVANCED_COPY_PATH.exists() and not OVERWRITE_ADVANCED:
        print(f'{ADVANCED_COPY_PATH} already exists. Reusing it; set OVERWRITE_ADVANCED = True to regenerate.')
    else:
        with ADVANCED_COPY_PATH.open('w') as fh:
            json.dump(advanced_cfg, fh, indent=2)
        print(f'Wrote {ADVANCED_COPY_PATH}')
    ADVANCED_JSON = ADVANCED_COPY_PATH
else:
    with ADVANCED_JSON.open() as fh:
        advanced_cfg = json.load(fh)
    print('Using existing advanced settings file without overrides.')

print('Advanced settings file in use:', ADVANCED_JSON)
summary_keys = ['enable_mpnn','use_multimer_design','design_models_override','sample_models','num_recycles_design','num_recycles_validation','soft_iterations','temporary_iterations','hard_iterations','greedy_iterations','max_trajectories','start_monitoring']
for key in summary_keys:
    value = advanced_cfg.get(key, '<not present>')
    print(f'{key:>28}: {value}')


Using existing advanced settings file without overrides.
Advanced settings file in use: /mnt/e/Code/BindCraft/bindcraft/settings_advanced/P51679_quick.json
                 enable_mpnn: True
         use_multimer_design: False
      design_models_override: [0]
               sample_models: False
         num_recycles_design: 1
     num_recycles_validation: 1
             soft_iterations: 28
        temporary_iterations: 16
             hard_iterations: 2
           greedy_iterations: 3
            max_trajectories: 5
            start_monitoring: 3


In [17]:
#Use a non-Qt backend. Switch matplotlib to Agg before importing anything that touches Qt:


import matplotlib
matplotlib.use("Agg")  # do this right after launching the kernel


## Launch BindCraft
Set `LAUNCH_RUN = True` when you are ready. The command runs in blocking mode; expect long runtimes for realistic designs. Interrupt the cell if you need to stop the run.

In [ ]:
LAUNCH_RUN = True
additional_args = []

env = os.environ.copy()
conda_prefix = env.get('CONDA_PREFIX')
if not conda_prefix:
    raise RuntimeError('Activate the BindCraft environment before launching.')
existing_ld = env.get('LD_LIBRARY_PATH', '')
if existing_ld:
    env['LD_LIBRARY_PATH'] = '{}/lib:{}'.format(conda_prefix, existing_ld)
else:
    env['LD_LIBRARY_PATH'] = '{}/lib'.format(conda_prefix)

cmd = [
    'python', '-u', str(BINDCRAFT_SCRIPT),
    '--settings', str(RUN_SETTINGS),
    '--filters', str(FILTERS_JSON),
    '--advanced', str(ADVANCED_JSON),
] + additional_args

print('Command to run:')
print(' '.join(shlex.quote(part) for part in cmd))
print('LD_LIBRARY_PATH:', env['LD_LIBRARY_PATH'])

if LAUNCH_RUN:
    result = subprocess.run(cmd, cwd=BINDCRAFT_ROOT, env=env, check=False)
    print(f'BindCraft finished with exit code {result.returncode}')
else:
    print('Set LAUNCH_RUN = True to start the design run.')


Command to run:
python -u /mnt/e/Code/BindCraft/bindcraft/bindcraft.py --settings /mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_quick.json --filters /mnt/e/Code/BindCraft/bindcraft/settings_filters/peptide_relaxed_filters.json --advanced /mnt/e/Code/BindCraft/bindcraft/settings_advanced/P51679_quick.json
LD_LIBRARY_PATH: /root/miniconda3/envs/BindCraft/lib
Available GPUs:
NVIDIA GeForce RTX 4070 Ti1: gpu
┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└────────

## Inspect recent outputs
Use the helper below to list the most recently modified files under your design directory once a run has started producing results.

In [ ]:
from datetime import datetime

if DESIGN_OUTPUT.exists():
    tracked_files = sorted(
        (path for path in DESIGN_OUTPUT.glob('**/*') if path.is_file()),
        key=lambda p: p.stat().st_mtime,
        reverse=True,
    )[:10]
    if tracked_files:
        print('Latest files:')
        for path in tracked_files:
            timestamp = datetime.fromtimestamp(path.stat().st_mtime).isoformat(sep=' ', timespec='seconds')
            print(f'{timestamp} :: {path.relative_to(DESIGN_OUTPUT)}')
    else:
        print('No files in the design output directory yet.')
else:
    print('Design output directory does not exist. Run the configuration cells first.')


No files in the design output directory yet.
